In [41]:
import os
import csv 
import maup
import geopandas as gpd
import matplotlib.pyplot as plt
from gerrychain import Graph
import networkx as nx
import pandas as pd
import numpy as np
from scipy.stats import t

In [42]:
YEAR = 2024

In [72]:
# Load and rename variables
# Washington 
shape_file = gpd.read_file('/Users/ppeaungvongpakdy/Downloads/district-shapes/POLYGON.shp')
shape_file['id'] = shape_file['id'].astype(int)
shape_file = shape_file.rename(columns = {'id': 'NAMELSAD'})
shape_file['NAMELSAD'] = shape_file['NAMELSAD'].apply(lambda x: f'Congressional District {x}')

#Arizona 
shape_file2 = gpd.read_file('/Users/ppeaungvongpakdy/Downloads/district-shapes-az/POLYGON.shp')
shape_file2['id'] = shape_file2['id'].astype(int)
shape_file2 = shape_file2.rename(columns = {'id': 'NAMELSAD'})
shape_file2['NAMELSAD'] = shape_file2['NAMELSAD'].apply(lambda x: f'Congressional District {x}')

<bound method NDFrame.head of                     NAMELSAD NAME  TotalPop  PopDevPct    DemPct    RepPct  \
0   Congressional District 1    1  770539.0   0.000014  0.610997  0.370255   
1   Congressional District 2    2  770531.0   0.000004  0.583336  0.399619   
2   Congressional District 3    3  770527.0  -0.000001  0.464523  0.520030   
3   Congressional District 4    4  770535.0   0.000009  0.397161  0.589598   
4   Congressional District 5    5  770526.0  -0.000003  0.437021  0.547182   
5   Congressional District 6    6  770523.0  -0.000007  0.557812  0.424242   
6   Congressional District 7    7  770521.0  -0.000009  0.849975  0.137500   
7   Congressional District 8    8  770512.0  -0.000021  0.499313  0.483818   
8   Congressional District 9    9  770536.0   0.000010  0.704295  0.281762   
9  Congressional District 10   10  770531.0   0.000004  0.559449  0.421161   

   WhitePct  MinorityPc  BlackPct  HispanicPc  PacificPct  AsianPct  \
0  0.629291    0.370709  0.034983    0.0

In [73]:
print(shape_file.head)

<bound method NDFrame.head of                     NAMELSAD NAME  TotalPop  PopDevPct    DemPct    RepPct  \
0   Congressional District 1    1  770539.0   0.000014  0.610997  0.370255   
1   Congressional District 2    2  770531.0   0.000004  0.583336  0.399619   
2   Congressional District 3    3  770527.0  -0.000001  0.464523  0.520030   
3   Congressional District 4    4  770535.0   0.000009  0.397161  0.589598   
4   Congressional District 5    5  770526.0  -0.000003  0.437021  0.547182   
5   Congressional District 6    6  770523.0  -0.000007  0.557812  0.424242   
6   Congressional District 7    7  770521.0  -0.000009  0.849975  0.137500   
7   Congressional District 8    8  770512.0  -0.000021  0.499313  0.483818   
8   Congressional District 9    9  770536.0   0.000010  0.704295  0.281762   
9  Congressional District 10   10  770531.0   0.000004  0.559449  0.421161   

   WhitePct  MinorityPc  BlackPct  HispanicPc  PacificPct  AsianPct  \
0  0.629291    0.370709  0.034983    0.0

In [74]:
print(shape_file2.head)

<bound method NDFrame.head of                    NAMELSAD NAME  TotalPop     PopDevPct    DemPct    RepPct  \
0  Congressional District 1    1  794611.0 -4.200000e-07  0.474840  0.509097   
1  Congressional District 2    2  794612.0  8.400000e-07  0.444203  0.533479   
2  Congressional District 3    3  794612.0  8.400000e-07  0.741595  0.235805   
3  Congressional District 4    4  794611.0 -4.200000e-07  0.517587  0.457929   
4  Congressional District 5    5  794612.0  8.400000e-07  0.395641  0.585030   
5  Congressional District 6    6  794611.0 -4.200000e-07  0.471336  0.509153   
6  Congressional District 7    7  794611.0 -4.200000e-07  0.655145  0.323339   
7  Congressional District 8    8  794610.0 -1.680000e-06  0.408037  0.573093   
8  Congressional District 9    9  794612.0  8.400000e-07  0.355689  0.627106   

   WhitePct  MinorityPc  BlackPct  HispanicPc  PacificPct  AsianPct  \
0  0.736484    0.263516  0.040145    0.138602    0.002844  0.054778   
1  0.600337    0.399663  0.

In [96]:
# Washington 
# Load voter data
csv_path = '/Users/ppeaungvongpakdy/Downloads/district-data/WA-district-data.csv'

# Read CSV file 
df = pd.read_csv(csv_path)
df = df[['ID', 'E_22_CONG_Total', 'E_22_CONG_Dem', 'E_22_CONG_Rep']]
df = df.drop(index=0)
df['Margins'] = (df['E_22_CONG_Dem'] - df['E_22_CONG_Rep']) / df['E_22_CONG_Total']
df = df.rename(columns = {'ID': 'NAMELSAD'})
df["STATEFP"] = 53

# Voter Power Calculation
SIGMA = 5
# t-pdf setup 
deg_f = 3 
t_dist = t(deg_f)

# For sanity checking
metamargin = 0 

Z = (df['Margins'] - metamargin) / SIGMA
num = t_dist.pdf(Z)
den = df['E_22_CONG_Total']

df['Voter Power'] = np.divide(num, den)

# Renaming variables and adding data
df['NAMELSAD'] = df['NAMELSAD'].apply(lambda x: f'Congressional District {x}')
df["STATEFP"] = 53

# Merging data
df = pd.merge(df, shape_file[['NAMELSAD', 'geometry']], on = 'NAMELSAD')

##########################################################################################################################################

#Arizona
# Load voter data
csv_path = '/Users/ppeaungvongpakdy/Downloads/district-data-az-cd/district-data.csv'

# Read CSV file 
df2 = pd.read_csv(csv_path)
df2 = df2[['ID', 'E_16-20_COMP_Total', 'E_16-20_COMP_Dem', 'E_16-20_COMP_Rep']]
df2 = df2.drop(index=0)
df2['Margins'] = (df2['E_16-20_COMP_Dem'] - df2['E_16-20_COMP_Rep']) / df2['E_16-20_COMP_Total']
df2 = df2.rename(columns = {'ID': 'NAMELSAD'})
df2["STATEFP"] = 4

# Voter Power Calculation
SIGMA = 5
# t-pdf setup 
deg_f = 3 
t_dist = t(deg_f)

# For sanity checking
metamargin = 0 

Z = (df2['Margins'] - metamargin) / SIGMA
num = t_dist.pdf(Z)
den = df2['E_16-20_COMP_Total']

df2['Voter Power'] = np.divide(num, den)

# Renaming variables and adding data
df2['NAMELSAD'] = df2['NAMELSAD'].apply(lambda x: f'Congressional District {x}')
df2["STATEFP"] = 4

# Merging data
df2 = pd.merge(df2, shape_file2[['NAMELSAD', 'geometry']], on = 'NAMELSAD')

In [97]:
df

,NAMELSAD,E_22_CONG_Total,E_22_CONG_Dem,E_22_CONG_Rep,Margins,STATEFP,Voter Power,geometry
0,Congressional District 1,286688,181994,104331,0.270897,53,0.000001,"POLYGON ((-122.11041 47.71820, -122.11040 47.7..."
1,Congressional District 2,337923,202980,134335,0.203138,53,0.000001,"POLYGON ((-123.18805 48.49897, -123.16002 48.4..."
2,Congressional District 3,319759,160314,157685,0.008222,53,0.000001,"POLYGON ((-122.34313 45.55982, -122.33907 45.5..."
3,Congressional District 4,226647,70710,150619,-0.352570,53,0.000002,"POLYGON ((-118.98895 46.15338, -118.99502 46.1..."
4,Congressional District 5,317006,127585,188648,-0.192624,53,0.000001,"POLYGON ((-118.36717 46.00071, -118.34633 46.0..."
5,Congressional District 6,347873,208710,138754,0.201096,53,0.000001,"POLYGON ((-122.83657 47.18750, -122.83434 47.1..."
6,Congressional District 7,346647,295998,49207,0.711938,53,0.000001,"POLYGON ((-122.32035 47.58611, -122.32125 47.5..."
7,Congressional District 8,336034,179001,155974,0.068526,53,0.000001,"POLYGON ((-121.97073 47.84622, -121.97383 47.8..."
8,Congressional District 9,239848,171746,67631,0.434087,53,0.000002,"POLYGON ((-122.16536 47.40140, -122.16533 47.4..."
9,Congressional District 10,267748,152544,114777,0.141054,53,0.000001,"POLYGON ((-123.15836 46.99587, -123.15847 46.9..."


In [98]:
df_concat_rows = pd.concat([df[['NAMELSAD', 'Margins', 'STATEFP', 'Voter Power', 'geometry']], df2[['NAMELSAD', 'Margins', 'STATEFP', 'Voter Power', 'geometry']]], axis=0)

In [99]:
df_concat_rows['Voter Power'] = ((df_concat_rows['Voter Power']  - df_concat_rows['Voter Power'].min()) / (df_concat_rows['Voter Power'].max() - df_concat_rows['Voter Power'].min()))*100

In [100]:
def round_to_nearest_tens(x):
    return round(x / 10) * 10

In [101]:
df_concat_rows['Voter Power'] = df_concat_rows['Voter Power'].apply(round_to_nearest_tens)

In [102]:
df_concat_rows

,NAMELSAD,Margins,STATEFP,Voter Power,geometry
0,Congressional District 1,0.270897,53,30,"POLYGON ((-122.11041 47.71820, -122.11040 47.7..."
1,Congressional District 2,0.203138,53,10,"POLYGON ((-123.18805 48.49897, -123.16002 48.4..."
2,Congressional District 3,0.008222,53,20,"POLYGON ((-122.34313 45.55982, -122.33907 45.5..."
3,Congressional District 4,-0.352570,53,60,"POLYGON ((-118.98895 46.15338, -118.99502 46.1..."
4,Congressional District 5,-0.192624,53,20,"POLYGON ((-118.36717 46.00071, -118.34633 46.0..."
5,Congressional District 6,0.201096,53,10,"POLYGON ((-122.83657 47.18750, -122.83434 47.1..."
6,Congressional District 7,0.711938,53,10,"POLYGON ((-122.32035 47.58611, -122.32125 47.5..."
7,Congressional District 8,0.068526,53,10,"POLYGON ((-121.97073 47.84622, -121.97383 47.8..."
8,Congressional District 9,0.434087,53,50,"POLYGON ((-122.16536 47.40140, -122.16533 47.4..."
9,Congressional District 10,0.141054,53,40,"POLYGON ((-123.15836 46.99587, -123.15847 46.9..."


In [103]:
# Save Data 
dir_path = os.path.dirname(os.path.realpath("__file__"))

path = os.path.join(dir_path, f'{YEAR}.house.VoterPower.csv')
df_concat_rows.to_csv(path, index=False, float_format='%.2f')